In [7]:
import os
from IPython.display import clear_output
from bs4 import BeautifulSoup
import math
import numpy as np
import shapely as shp
import pandas as pd
import geopandas as gpd
# import matplotlib.pyplot as plt
import rasterio as rio
import rasterio.mask

### Manual Input

In [8]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [9]:
# Spatial homogeneity ROIs
distances = [30,100,300,600,900]

In [10]:
df_Site = pd.read_excel(cwd + "\\Site - With Moved ROI (Coordinates Only).xlsx")
df_Site.head()

,Number,Site,Latitude,Longitude,Reference network
0,1,ATGE,52.466778,12.959778,HYPERNET
1,2,ATLAS-Mohammed V,33.406152,-5.103319,Other
2,3,ATLAS-Mohammed V New,33.404814,-5.101614,Other
3,4,AT-Mmg,47.316700,10.970300,FLOX
4,5,BASP,39.049139,-2.075917,HYPERNET


### Real work begins

In [12]:
for i in range(df_Site.shape[0]):
    site_Name = df_Site["Site"][i]
    print(f"{site_Name} starts!")
    site_Lat = df_Site.loc[i,"Latitude"]
    site_Lon = df_Site.loc[i,"Longitude"]
    if site_Name[-3:] != 'New':
        # Find the paths to images and metadata .xml files
        # L1C B08
        for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
            for name in files:
                temp = os.path.join(path, name)
                if "IMG_DATA" in temp and temp[-3:] == 'jp2' and "B08" in temp:
                    path_L1C_B08_raw = temp
        # L1C metadata MTD_DS.xml
        for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
            for name in files:
                temp = os.path.join(path, name)
                if "MTD_DS.xml" in temp:
                    path_L1C_xml_DS = temp
        # L1C metadata MTD_TL.xml
        for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L1C"):
            for name in files:
                temp = os.path.join(path, name)
                if "MTD_TL.xml" in temp:
                    path_L1C_xml_TL = temp
        # L2A B04 & B08
        for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L2A"):
            for name in files:
                temp = os.path.join(path, name)
                if temp[-3:] == 'jp2'in temp and "10m" in temp and "B04" in temp :
                    path_L2A_B04_raw = temp
                if temp[-3:] == 'jp2'in temp and "10m" in temp and "B08" in temp :
                    path_L2A_B08_raw = temp
        # L2A metadata MTD_DS.xml
        for path, subdirs, files in os.walk(cwd_Images_Raw + "\\" + site_Name + "\\L2A"):
            for name in files:
                temp = os.path.join(path, name)
                if "MTD_DS.xml" in temp:
                    # print(temp)
                    path_L2A_xml_DS = temp

        # Read raster images
        image_L1C_B08 = rio.open(path_L1C_B08_raw)
        image_L2A_B04 = rio.open(path_L2A_B04_raw)
        image_L2A_B08 = rio.open(path_L2A_B08_raw)
        values_L1C_B08 = image_L1C_B08.read(1)
        values_L2A_B04 = image_L2A_B04.read(1)
        values_L2A_B08 = image_L2A_B08.read(1)
        crs_L1C = image_L1C_B08.crs.data["init"].split(":")[1]
        crs_L2A = image_L2A_B04.crs.data["init"].split(":")[1]

        # Parse metadata .xml files
        # Read the DS xml file of L2A
        with open(path_L2A_xml_DS, 'r') as f:
            data = f.read()
        BS_L2A_dS = BeautifulSoup(data, "xml")
        # Get the quantification value! 
        quantification_L2A = int(BS_L2A_dS.find("BOA_QUANTIFICATION_VALUE").text)
        # Get the radiometric offset!
        offset_L2A_B04 = int(BS_L2A_dS.find("BOA_ADD_OFFSET", {"band_id": "3"}).text)
        offset_L2A_B08 = int(BS_L2A_dS.find("BOA_ADD_OFFSET", {"band_id": "7"}).text)
        # Read the DS xml file of L1C
        with open(path_L1C_xml_DS, 'r') as f:
            data = f.read()
        BS_L1C_dS = BeautifulSoup(data, "xml")
        # Get the quantification value! 
        quantification_L1C = int(BS_L1C_dS.find("QUANTIFICATION_VALUE").text)
        # Get the radiometric offset!
        offset_L1C = int(BS_L1C_dS.find("RADIO_ADD_OFFSET", {"band_id": "7"}).text)
        # Get the U
        U_L1C = float(BS_L1C_dS.find("U").text)
        # Get the solar irradiance
        SolarIrr = float(BS_L1C_dS.find("SOLAR_IRRADIANCE", {"bandId": "7"}).text)
        # Read the TL xml file of L1C
        with open(path_L1C_xml_TL, 'r') as f:
            data = f.read()
        BS_L1C_dS = BeautifulSoup(data, "xml")
        # Get the sun zenith angle! There should be a 23 x 23 arrays in the xml. Now we save each row as an array and keep all these arrays into a list
        list_SunZenith = []
        for row in BS_L1C_dS.find("Sun_Angles_Grid").find("Zenith").find_all("VALUES"):
            temp_List = row.text.split(" ")
            temp_Arr = np.array(temp_List)
            temp_Arr = temp_Arr.astype(float)
            list_SunZenith.append(temp_Arr)
        # Now we stack these nested-in-list arrays into a 2d array
        index = 0
        for arr in list_SunZenith:
            if index == 0:
                arr_SunZenith = arr
            else:
                arr_SunZenith = np.vstack((arr_SunZenith, arr))
            index = index + 1
        # Get the shape of L1C image, which should be (10980, 10980)
        shape_L1C = values_L1C_B08.shape
        # Repeat each element of sun zenith angle array, in both axies. The final array should have a shape of (11500, 11500)
        arr_SunZenith_Repeat = np.repeat(arr_SunZenith, 500, axis = 1)
        arr_SunZenith_Repeat = np.repeat(arr_SunZenith_Repeat, 500, axis = 0)
        # Index only the first 10980 of each dimension
        arr_SunZenith_Assigned = arr_SunZenith_Repeat[0:shape_L1C[0], 0:shape_L1C[1]]

        # Calculate NDVI
        NDVI = ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A - (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A) / ((values_L2A_B08 + offset_L2A_B08).astype(float) / quantification_L2A + (values_L2A_B04 + offset_L2A_B04).astype(float) / quantification_L2A )
        src = image_L2A_B04
        out_meta = src.meta
        out_meta.update({
            "driver": "GTiff",
            "dtype": 'float64'
        })
        with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NDVI.tif", 'w', **out_meta) as dest:
            dest.write(NDVI, 1)

        # Calculate radiance of B8 of L1C
        # radiance = reflectance * cos(radians(SunZenithAngle)) * solarIrradiance * U / pi
        radiance = (values_L1C_B08 + offset_L1C).astype(float)  * np.cos(np.radians(arr_SunZenith_Assigned)) * SolarIrr / quantification_L1C / (math.pi * (1 / U_L1C))
        NIRv = NDVI * radiance
        src = image_L1C_B08
        out_meta = src.meta
        out_meta.update({
            "driver": "GTiff",
            "dtype": 'float64'
        })
        with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv.tif", 'w', **out_meta) as dest:
            dest.write(NIRv, 1)
        
        # Re-open NIRv.tif for clipping
        src = rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv.tif")
        print(f"Calculation of NDVI and NIRv of site {site_Name} finished!")
    else:
        # If a site is a moved ROI, then we directly open the NIRv.tif of original site!
        src = rio.open(cwd_Images_Processed + "\\" + site_Name[:-4] + "\\NIRv.tif")
        print(f"Original NIRv of site {site_Name} has been read!")

    # Read shapefiles!
    list_GDF = []
    for distance in distances:
        temp_gdf = gpd.read_file(cwd_Images_Processed + "\\" + site_Name + "\\" + str(distance) + "m.shp")
        list_GDF.append(temp_gdf)
    
    for index in range(len(list_GDF)):
        temp_Distance = distances[index]
        temp_GDF = list_GDF[index]
        out_image, out_transform = rio.mask.mask(src, temp_GDF.geometry, crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                        "height": out_image.shape[1],
                        "width": out_image.shape[2],
                        "transform": out_transform})

        with rio.open(cwd_Images_Processed + "\\" + site_Name + "\\NIRv " + str(temp_Distance) + ".tif", "w", **out_meta) as dest:
            dest.write(out_image)
    print(site_Name + " finished!")
    clear_output(wait=True)

WWUK starts!
WWUK finished!
